In [13]:
from datasets import load_dataset, Dataset, DatasetDict, load_from_disk
from bigdata_a3_utils import download_all_amazon_reviews, load_compressed_dataset
from pathlib import Path
import polars as pl
import pandas as pd
from itertools import islice
from typing import Generator

In [99]:
auto_review: DatasetDict = load_dataset(
    "McAuley-Lab/Amazon-Reviews-2023",
    "raw_review_Automotive",
    trust_remote_code=True,
    streaming=True,
)

auto_meta: DatasetDict = load_dataset(
    "McAuley-Lab/Amazon-Reviews-2023",
    "raw_meta_Automotive",
    trust_remote_code=True,
    streaming=True,
)

In [100]:
review_dataset: Dataset = auto_review["full"]
meta_dataset: Dataset = auto_meta["full"]

In [101]:
def stream_to_polars(dataset: Dataset, batch_size=100_000, total=1_000_000) -> pl.DataFrame:
    batches: list[pl.DataFrame] = []
    it: Generator = iter(dataset)
    for _ in range(0, total, batch_size):
        chunk = list(islice(it, batch_size))
        if not chunk:
            break
        batches.append(pl.DataFrame(chunk, infer_schema_length=1000))
    return pl.concat(batches, how="vertical")

In [102]:
df_review: pl.DataFrame = stream_to_polars(review_dataset, batch_size=1_000, total=1_000)
df_meta: pl.DataFrame = stream_to_polars(meta_dataset, batch_size=1_000, total=1_000)
# df = pl.DataFrame(list(islice(meta_dataset, 10)))

In [103]:
# df_review.head()
# df_meta.head()
merged_df: pl.DataFrame = df_review.join(df_meta, on="parent_asin", how="left")
merged_df

rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,main_category,title_right,average_rating,rating_number,features,description,price,images_right,videos,store,categories,details,bought_together,subtitle,author
f64,str,str,list[struct[4]],str,str,str,i64,i64,bool,str,str,f64,i64,list[str],list[str],str,struct[4],struct[3],str,list[str],str,null,str,null
5.0,"""It fit our 2012 Chevy Colorado…","""Item came as described! It fit…",[],"""B01LZA8SGZ""","""B0BV88374L""","""AGXVBIUFLFGMVLATYXHJYL4A5Q7Q""",1513092936205,0,true,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
5.0,"""Easy to put on!!""","""Ease of application process. C…",[],"""B0B2WGS5ND""","""B0B2WGS5ND""","""AFE337D2J37YRU5U6MVTVKNDKWDA""",1659124303053,0,true,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
5.0,"""Perfect""","""Nice quality, framed in silver…",[],"""B00A0GV20Q""","""B00A0GV20Q""","""AEVWAM3YWN5URJVJIZZ6XPD2MKIA""",1388666735000,0,true,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2.0,"""Not waterproof""","""Description said it is waterpr…",[],"""B08C27WWVG""","""B08C27WWVG""","""AHITBJSS7KYUBVZPX7M2WJCOIVKQ""",1651801619265,0,true,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
5.0,"""Very nice""","""Looks great on my son’s 370Z""",[],"""B0719J5ZNY""","""B0719J5ZNY""","""AHITBJSS7KYUBVZPX7M2WJCOIVKQ""",1574097084236,0,true,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
5.0,"""Easy install""","""Wasn’t sure if I’d like it but…",[],"""B07BSDST3X""","""B07BSDST3X""","""AEZHVVKEZAY6MAP5WA62MAYTMHWQ""",1565781661279,0,true,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
5.0,"""Quality""","""Soft, quality material, feels …",[],"""B01H2GNFRO""","""B01H2GNFYM""","""AEZHVVKEZAY6MAP5WA62MAYTMHWQ""",1563743892185,0,true,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
5.0,"""Quality product""","""Love this, it’s by far my favo…",[],"""B0772Y2KNJ""","""B0772Y2KNJ""","""AEZHVVKEZAY6MAP5WA62MAYTMHWQ""",1554246138438,1,true,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [ ]:
# buffer_size = 100_000
# stream = iter(dataset)

# while True:
#     batch = list(islice(stream, buffer_size))
#     if not batch:
#         break
#     print(batch[0])  # example

In [ ]:
# save_path = Path("data/raw/raw_review_Automotive")
# # auto_review.save_to_disk(str(save_path))

In [ ]:
# loaded_review: DatasetDict = load_from_disk(str(save_path))

In [ ]:
# subset: Dataset = loaded_review["full"]
# subset

In [ ]:
# df = pl.DataFrame(subset[0:9_955_450])
# df = pl.DataFrame(subset[0:19_955_450])

In [ ]:
# df